In [2]:
tf.test.is_gpu_available()
tf.config.list_physical_devices('GPU')

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
blah = 10000

In [1]:
from importlib import reload
import numpy as np
import os
import copy
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import matplotlib.pyplot as plt
from scipy.sparse import csc_matrix, csr_matrix
import pickle
import json
import pretty_midi
import sys
from collections import namedtuple
import timeit
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.callbacks import LearningRateScheduler
# import my modules
import src.midi_utils as midi_utils
import src.data as data
import src.models as models
import src.ml_classes as ml_classes
import src.exp_utils as exp_utils
import src.losses as losses


In [2]:
# data params
model_inputs = ['Pn', 'TBn', 'TSBn']
model_outputs = ['Vn']
seq_length = 50
sub_beats = 2
example_bars_skip = 4
use_base_key = False
transpose = False
st = 0
nth_file = 15
vel_cutoff = 6
data_folder_prefix = '_8'

##### Model Config ####
### general network params
hidden_state = 200
recurrent_dropout=0.0

### encoder params
bi_encoder_lstms = 2
uni_encoder_lstms = 1
conv = False
ar_inputs = None
# pitch_stride = 6
# conv = {'F_n': [32, 32, 48, 48, 48, 24], # number of filters
#         'F_s': [(8,12), (4,4), (4,4), (4,4), (4,4), (4,4)], # size of filters
#         'strides': [(1, pitch_stride), (1, 1), (2, 1), (2,1), (2,1), (2,2)],  # strides
#         'batch_norm': True # apply batch norm after each conv operation (after activation)
#         }

##### Training Config ####
batch_size = 64
lr = 0.001
lr_decay_rate = 0.3**(1/1500)
min_lr = 0.00005
epochs = 2
monitor = 'loss'
loss_weights = None
clipvalue = 1
loss = 'mse'
metrics = ['accuracy', 'mse']

# musicvae used 48 free bits for 2-bars, 256 for 16 bars (see https://arxiv.org/pdf/1803.05428.pdf)
# Variational specific parameters
max_beta = 3
beta_rate = 0.2**(1/1000) # at 1000 epochs, we want (1 - something) * max_beta
free_bits=0
kl_weight = 1

#other
continue_run = None
log_tensorboard = False
ar_inc_batch_shape = False # sometimes needed to make training work...

In [20]:
# import modules, including a reload statement so that they can be reimported after a change to the methods 
import src.midi_utils as midi_utils
reload(midi_utils)

import src.data as data
reload(data)

import src.models as models
reload(models)

import src.ml_classes as ml_classes
reload(ml_classes)

<module 'src.ml_classes' from '/storage/781-piano-autoencoder/src/ml_classes.py'>

In [12]:
path = 'test/'
no = 0
model_datas_train, seconds = data.folder2nbq('training_data/midi_train' + data_folder_prefix, 
                                            return_ModelData_object=True,
                                            seq_length=seq_length, 
                                            sub_beats=sub_beats, 
                                            example_bars_skip=example_bars_skip, 
                                            use_base_key=use_base_key, 
                                            nth_file=nth_file, 
                                            vel_cutoff=vel_cutoff)

model_datas_val, seconds = data.folder2nbq('training_data/midi_val' + data_folder_prefix, 
                                        return_ModelData_object=True,
                                        seq_length=seq_length, 
                                        sub_beats=sub_beats, 
                                        example_bars_skip=example_bars_skip, 
                                        use_base_key=use_base_key, 
                                        vel_cutoff=vel_cutoff)


model_input_reqs, model_output_reqs = models.get_model_reqs(model_inputs, model_outputs, sub_beats=sub_beats)

callbacks = []
# train loss model checkpoint
callbacks.append(tf.keras.callbacks.ModelCheckpoint(path + f'{no}_best_train_weights.hdf5',
                            monitor='loss', verbose=1, save_best_only=True, save_weights_only=True))
# val loss model checkpoint
callbacks.append(tf.keras.callbacks.ModelCheckpoint(path + f'{no}_best_val_weights.hdf5',
                            monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True))
# early stopping, if needed
callbacks.append(tf.keras.callbacks.EarlyStopping(monitor=monitor, min_delta=0, patience=15))
# learning rate scheduler
callbacks.append(LearningRateScheduler(exp_utils.decay_lr(min_lr, lr_decay_rate)))
# log to tensorboard
if log_tensorboard:
    callbacks.append(tf.keras.callbacks.TensorBoard(log_dir='experiments/tb/', histogram_freq = 1))

# model kwargs - for the encoder/decoder builder functions, make a dictionary to pass as kwargs
model_kwargs = {# general model parameters
                'recurrent_dropout':recurrent_dropout,
                'hidden_state':hidden_state,
                'seq_length':seq_length,
                'uni_encoder_lstms':uni_encoder_lstms,
                'bi_encoder_lstms':bi_encoder_lstms,
                'conv':conv,

                # decoder parameters
                'ar_inputs':ar_inputs,
                'batch_size':batch_size, # not used in encoder, currently...
                'ar_inc_batch_shape':ar_inc_batch_shape,
                'conv':conv,
                }

inputs_tf, pred = models.create_nbq_bi_graph(model_input_reqs, model_output_reqs, **model_kwargs)


model = tf.keras.Model(inputs=inputs_tf, outputs=pred, name=f'bi_uni_model')
model.summary()





# save a plot of the model
# tf.keras.utils.plot_model(seq_model, to_file=f'{path}model_plot.png')

dg = ml_classes.ModelDataGenerator([md for md in model_datas_train.values()],
                                    [model_in.name for model_in in model_input_reqs if model_in.md],
                                    [model_out.name for model_out in model_output_reqs if model_out.md],
                                    t_force=True, batch_size = batch_size, seq_length=seq_length,
                                    sub_beats=sub_beats, V_no_zeros=False)

dg_val = ml_classes.ModelDataGenerator([md for md in model_datas_val.values()],
                                    [model_in.name for model_in in model_input_reqs if model_in.md],
                                    [model_out.name for model_out in model_output_reqs if model_out.md],
                                    t_force=True, batch_size = batch_size, seq_length=seq_length,
                                    sub_beats=sub_beats, V_no_zeros=False)

opt = tf.keras.optimizers.Adam(learning_rate=lr, clipvalue=clipvalue)



model.compile(optimizer=opt, loss=loss, metrics=metrics, loss_weights=loss_weights)


if continue_run != None:
    model.load_weights(f'experiments/run_{continue_run}/{continue_run}_best_train_weights.hdf5')

history = model.fit(dg, validation_data=dg_val, epochs=epochs, callbacks=callbacks, verbose=2)


# save the model history
with open(f'{path}history-{epochs}epochs.json', 'w') as f:
    json.dump(str(history.history), f)

# add weights to sacred... Or not, they can exceed max size! 
# exp_utils.capture_weights(_run)


  8%|▊         | 1/12 [00:00<00:01,  7.51it/s]

00h 43m 51s of data
created model data TSn :    (50,) data shape,     278 training examples
created model data TEn :    (50,) data shape,     278 training examples
created model data TBn :    (50, 4) data shape,     278 training examples
created model data TMn :    (50, 16) data shape,     278 training examples
created model data TSBn :    (50, 2) data shape,     278 training examples
created model data Pn :    (50, 88) data shape,     278 training examples
created model data PSn :    (50, 1) data shape,     278 training examples
created model data PCn :    (50, 12) data shape,     278 training examples
created model data Vn :    (50, 1) data shape,     278 training examples
created model data tempo :    (1,) data shape,     278 training examples
created model data key :    (12,) data shape,     278 training examples
created model data V_mean :    (1,) data shape,     278 training examples


100%|██████████| 12/12 [00:03<00:00,  3.17it/s]


00h 38m 47s of data
created model data TSn :    (50,) data shape,     250 training examples
created model data TEn :    (50,) data shape,     250 training examples
created model data TBn :    (50, 4) data shape,     250 training examples
created model data TMn :    (50, 16) data shape,     250 training examples
created model data TSBn :    (50, 2) data shape,     250 training examples
created model data Pn :    (50, 88) data shape,     250 training examples
created model data PSn :    (50, 1) data shape,     250 training examples
created model data PCn :    (50, 12) data shape,     250 training examples
created model data Vn :    (50, 1) data shape,     250 training examples
created model data tempo :    (1,) data shape,     250 training examples
created model data key :    (12,) data shape,     250 training examples
created model data V_mean :    (1,) data shape,     250 training examples
Model: "bi_uni_model"
___________________________________________________________________________

In [110]:
pred_tf = model.predict(random_examples)
# find axis that corresponds to velocity
v_index = np.where(np.array(model.output_names) == 'Vn_out')[0][0]
print('velocity index:', v_index)
model_datas_pred_tf = copy.deepcopy(model_datas_val)
model_datas_pred_tf['Vn'].data[idx,...] = np.array(pred_tf)

velocity index: 0


In [108]:
np.array(pred_tf)[v_index,...].shape

(50, 1)

In [79]:
model_kwargs.update({'stateful': True})
reqs_tf = models.create_nbq_bi_graph(model_input_reqs, model_output_reqs, **model_kwargs)

encoder = tf.keras.Model(inputs=reqs_tf['encoder_input'], outputs=reqs_tf['encoder_output'], name=f'encoder')
decoder = tf.keras.Model(inputs=reqs_tf['decoder_input'], outputs=reqs_tf['decoder_output'], name=f'decoder')

models.load_weights_safe(encoder, path + f'{no}_best_val_weights.hdf5', by_name=True)
models.load_weights_safe(decoder, path + f'{no}_best_val_weights.hdf5', by_name=True)


weights loaded successfully
weights loaded successfully


In [80]:
random_examples, idx = data.n_rand_examples(model_datas_val, n=batch_size)
v_index = np.where(np.array(model.output_names) == 'Vn_out')[0][0]
print('velocity index:', v_index)
random_examples['encoded'] = encoder.predict(random_examples)

velocity index: 0


In [105]:
pred = np.zeros((batch_size, seq_length))

Vn_out = np.zeros((batch_size,1,1))
for i in range(seq_length):
    step_input = {'encoded': random_examples['encoded'][:,i,:][:,None,:], 'Vn_ar': Vn_out}
#     step_input = {'encoded': np.zeros((64,1,400)), 'Vn_ar': Vn_out}
    Vn_out = decoder.predict(step_input, batch_size=batch_size)
    pred[:,i] = Vn_out.flatten()

model_datas_pred = copy.deepcopy(model_datas_val)
model_datas_pred['Vn'].data[idx,...] = pred[...,None]
os.mkdir(path + 'midi/')
for i in idx:
    mds_orig = {md.name: md.data[i] for _, md in model_datas_val.items()}
    mds_pred = {md.name: md.data[i] for _, md in model_datas_pred.items()}
    pm_original = data.nbq2pm(mds_orig)
    pm_pred = data.nbq2pm(mds_pred)
    pm_original.write(path + 'midi/' + f'ex{i}original.mid')
    pm_pred.write(path + 'midi/' + f'ex{i}prediction_teacher_forced.mid')

FileExistsError: [Errno 17] File exists: 'test/midi/'

In [95]:
np.array(pred).shape
Vn_out.shape

(64, 1, 1)

In [100]:
pred = np.zeros((batch_size, seq_length))
pred[:,0] = Vn_out.flatten()
pred

array([[0.35264522, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.35529041, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.35734826, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.35198653, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.35377997, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.35339791, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [65]:
x = tf.keras.Input(shape=(1,hidden_state * 2), name=f'encoded')
[64] + x.shape[1:]

TensorShape([64, 1, 400])

In [78]:
# import modules, including a reload statement so that they can be reimported after a change to the methods 
import src.midi_utils as midi_utils
reload(midi_utils)

import src.data as data
reload(data)

import src.models as models
reload(models)

import src.ml_classes as ml_classes
reload(ml_classes)

<module 'src.ml_classes' from '/storage/781-piano-autoencoder/src/ml_classes.py'>